In [2]:
from io import StringIO
import pandas as pd
from sqlalchemy import create_engine, text
engine = create_engine('sqlite://', echo=False)
import sqlite3
import contextlib

def df_to_database(df, name):
    """
       Deze functie maakt een database van een dataframe.
       Parameters:
                  df  = de dataframe waarvan je een database tabel wil maken.
                         
                  name = De naam die je de database wil geven.
                         Zorg ervoor dat deze parameter een string is.
    """
    df.to_sql(name, con=engine)
    
    display(df)

def string_to_df(string):
    """
       Deze functie maakt van een string een dataframe,
       zolang het standaard CSV formaat wordt aangehouden.
       Parameters:
                  string = de string waarvan je een dataframe wil maken.
    """
    return pd.read_csv(StringIO(string))

def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """

    query_output = engine.execute(query).fetchall()
    column_names = engine.execute(query)._metadata.keys
    return pd.DataFrame(query_output, columns = column_names)

In [3]:
sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()

In [4]:
query_r = "SELECT * FROM sqlite_sequence" # read query

sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()
df = pd.read_sql_query(query_r,sql_connect)
df

,name,seq
0,Team,103916
1,Country,51958
2,League,51958
3,Match,51958
4,Player,11075
5,Player_Attributes,183978
6,Team_Attributes,1458


In [5]:
import contextlib
@contextlib.contextmanager
def db_read_only(file):
    sql_connect = sqlite3.connect('file:'+file+'?mode=ro',uri=True)
    cursor = sql_connect.cursor()
    yield sql_connect
    sql_connect.commit()
    sql_connect.close() 

In [6]:
file = 'database.sqlite'
with db_read_only(file) as ro:
    data = pd.read_sql_query(query_r, ro)

In [7]:
data['name']

0                 Team
1              Country
2               League
3                Match
4               Player
5    Player_Attributes
6      Team_Attributes
Name: name, dtype: object

In [8]:
df_list = []
with db_read_only(file) as ro:
    for table in data['name']:
        query_r = 'SELECT * FROM ' + str(table)
        data = pd.read_sql_query(query_r, ro)
        df_list.append(data)
df_list[0]

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [9]:
team = df_list[0]
country = df_list[1]
league = df_list[2]
match = df_list[3]
player = df_list[4]
player_attributes = df_list[5]
team_attributes = df_list[6]

In [10]:
match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
match = match.merge(team, left_on = "home_team_api_id", right_on = "team_api_id", how = "left", suffixes = ["_match", "_team"])
match

,id_match,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_team,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,3.25,4.00,1.73,3.40,4.20,1,9987,673.0,KRC Genk,GEN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,3.75,1.91,3.25,3.60,3,10000,15005.0,SV Zulte-Waregem,ZUL
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,3.20,2.50,2.30,3.20,2.75,5,9984,1750.0,KSV Cercle Brugge,CEB
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,3.75,5.50,1.44,3.75,6.50,7,9991,674.0,KAA Gent,GEN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,3.50,1.65,4.75,3.30,1.67,9,7947,NaN,FCV Dender EH,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,49479,10190,898.0,FC St. Gallen,GAL
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,49122,9824,286.0,FC Vaduz,VAD
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,49115,9956,322.0,Grasshopper Club Zürich,GRA
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,51606,7896,NaN,Lugano,LUG


In [12]:
df_to_database(match, 'match_sql')
print('Tabel 1: match_sql')

,id_match,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_team,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,3.25,4.00,1.73,3.40,4.20,1,9987,673.0,KRC Genk,GEN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,3.75,1.91,3.25,3.60,3,10000,15005.0,SV Zulte-Waregem,ZUL
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,3.20,2.50,2.30,3.20,2.75,5,9984,1750.0,KSV Cercle Brugge,CEB
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,3.75,5.50,1.44,3.75,6.50,7,9991,674.0,KAA Gent,GEN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,3.50,1.65,4.75,3.30,1.67,9,7947,NaN,FCV Dender EH,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,49479,10190,898.0,FC St. Gallen,GAL
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,49122,9824,286.0,FC Vaduz,VAD
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,49115,9956,322.0,Grasshopper Club Zürich,GRA
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,51606,7896,NaN,Lugano,LUG


Tabel 1: match_sql


In [13]:
def total_games(team):

    query = f"""

    SELECT COUNT(match_api_id) AS amount_of_games, season
    FROM match_sql
    WHERE (home_team_api_id = {team} OR away_team_api_id = {team})
    GROUP BY season
        
    """

    return show_query(query)

In [14]:
result = total_games(9906)
result

,amount_of_games,season
0,38,2008/2009
1,38,2009/2010
2,38,2010/2011
3,38,2011/2012
4,38,2012/2013
5,38,2013/2014
6,38,2014/2015
7,38,2015/2016


In [15]:
def season_2010(team):
    query = f"""

    SELECT COUNT(match_api_id) AS amount_of_games, season
    FROM match_sql
    WHERE (home_team_api_id = {team} OR away_team_api_id = {team})
    AND date LIKE '%2010%'
    GROUP BY season
    
    """

    return show_query(query)

In [16]:
result = season_2010(9906)
result

,amount_of_games,season
0,23,2009/2010
1,16,2010/2011


In [17]:
query = """

SELECT *
FROM match_sql

"""

show_query(query)

,index,id_match,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,GBD,GBA,BSH,BSD,BSA,id_team,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,...,3.25,4.00,1.73,3.40,4.20,1,9987,673.0,KRC Genk,GEN
1,1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,...,3.25,3.75,1.91,3.25,3.60,3,10000,15005.0,SV Zulte-Waregem,ZUL
2,2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,...,3.20,2.50,2.30,3.20,2.75,5,9984,1750.0,KSV Cercle Brugge,CEB
3,3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,...,3.75,5.50,1.44,3.75,6.50,7,9991,674.0,KAA Gent,GEN
4,4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,...,3.50,1.65,4.75,3.30,1.67,9,7947,NaN,FCV Dender EH,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,...,NaN,NaN,NaN,NaN,NaN,49479,10190,898.0,FC St. Gallen,GAL
25975,25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,...,NaN,NaN,NaN,NaN,NaN,49122,9824,286.0,FC Vaduz,VAD
25976,25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,...,NaN,NaN,NaN,NaN,NaN,49115,9956,322.0,Grasshopper Club Zürich,GRA
25977,25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,...,NaN,NaN,NaN,NaN,NaN,51606,7896,NaN,Lugano,LUG


# DIT IS DE GOEDE

In [18]:
import pandas as pd
import sqlite3

def ranking(league, season):
    query = f"""WITH homepoints AS (SELECT home_team_api_id AS team, team_long_name, season,
                           CASE
                             WHEN home_team_goal > away_team_goal THEN 3
                             WHEN home_team_goal = away_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql
                        WHERE league_id = {league}
                        AND season = '{season}'),
                awaypoints AS (SELECT away_team_api_id AS team, team_long_name, season,
                           CASE
                             WHEN away_team_goal > home_team_goal THEN 3
                             WHEN away_team_goal = home_team_goal THEN 1
                             ELSE 0
                           END AS points
                        FROM match_sql
                        WHERE league_id = {league}
                        AND season = '{season}'),
                allpoints AS (SELECT team, team_long_name, season, SUM(points) AS points
                              FROM homepoints
                              GROUP BY team
                              UNION ALL
                              SELECT team, team_long_name, season, SUM(points) AS points
                              FROM awaypoints
                              GROUP BY team)
                        
    SELECT team, team_long_name, season, SUM(points) AS points
    FROM allpoints
    GROUP BY team
    ORDER BY points DESC
    """
    df = show_query(query)
    return df.set_index(pd.Index([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]))

In [19]:
result1 = ranking(21518, '2008/2009')
result1

,team,team_long_name,season,points
1,8634,FC Barcelona,2008/2009,87
2,8633,Real Madrid CF,2008/2009,78
3,8302,Sevilla FC,2008/2009,70
4,9906,Atlético Madrid,2008/2009,67
5,10205,Villarreal CF,2008/2009,65
6,10267,Valencia CF,2008/2009,62
7,9783,RC Deportivo de La Coruña,2008/2009,58
8,9864,Málaga CF,2008/2009,55
9,8661,RCD Mallorca,2008/2009,51
10,8558,RCD Espanyol,2008/2009,47


In [20]:
def our_place(team):
    new = result1[result1["team"] == team]
    return new

In [21]:
our_place(9906)

,team,team_long_name,season,points
4,9906,Atlético Madrid,2008/2009,67


In seizoen 2008/2009 zijn wij 4e geworden.

In [22]:
def ranking_total(league):
    query = f"""WITH homepoints AS (SELECT home_team_api_id AS team, team_long_name, season,
                               CASE
                                 WHEN home_team_goal > away_team_goal THEN 3
                                 WHEN home_team_goal = away_team_goal THEN 1
                                 ELSE 0
                               END AS points
                            FROM match_sql
                            WHERE league_id = {league}),
                    awaypoints AS (SELECT away_team_api_id AS team, team_long_name, season,
                               CASE
                                 WHEN away_team_goal > home_team_goal THEN 3
                                 WHEN away_team_goal = home_team_goal THEN 1
                                 ELSE 0
                               END AS points
                            FROM match_sql
                            WHERE league_id = {league}),
                    allpoints AS (SELECT team, team_long_name, season, SUM(points) AS points
                                  FROM homepoints
                                  GROUP BY team
                                  UNION ALL
                                  SELECT team, team_long_name, season, SUM(points) AS points
                                  FROM awaypoints
                                  GROUP BY team)

    SELECT team, team_long_name, SUM(points) AS points
    FROM allpoints
    GROUP BY team
    ORDER BY points DESC
    """
    return show_query(query)

In [23]:
final = ranking_total('21518')
final

,team,team_long_name,points
0,8634,FC Barcelona,745
1,8633,Real Madrid CF,720
2,9906,Atlético Madrid,560
3,10267,Valencia CF,500
4,8302,Sevilla FC,482
5,8315,Athletic Club de Bilbao,437
6,10205,Villarreal CF,407
7,9864,Málaga CF,396
8,8558,RCD Espanyol,364
9,8305,Getafe CF,353


Als we zouden kijken naar het totaal van alle seizoenen staan wij op plek 3.

In [36]:
new_team = team_attributes.merge(result1, left_on = "team_api_id", right_on = "team")
new_team

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team,team_long_name,season,points
0,40,1861,9865,2010-02-22 00:00:00,65,Balanced,NaN,Little,45,Mixed,...,High,65,Press,30,Narrow,Cover,9865,UD Almería,2008/2009,46
1,41,1861,9865,2011-02-22 00:00:00,55,Balanced,NaN,Little,53,Mixed,...,Medium,68,Double,67,Wide,Cover,9865,UD Almería,2008/2009,46
2,42,1861,9865,2012-02-22 00:00:00,42,Balanced,NaN,Little,70,Long,...,Medium,53,Press,45,Normal,Cover,9865,UD Almería,2008/2009,46
3,43,1861,9865,2013-09-20 00:00:00,46,Balanced,NaN,Little,65,Mixed,...,Medium,53,Press,45,Normal,Cover,9865,UD Almería,2008/2009,46
4,44,1861,9865,2014-09-19 00:00:00,46,Balanced,41.0,Normal,65,Mixed,...,Medium,53,Press,45,Normal,Cover,9865,UD Almería,2008/2009,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,1337,483,10205,2011-02-22 00:00:00,42,Balanced,NaN,Little,41,Mixed,...,Medium,48,Press,54,Normal,Cover,10205,Villarreal CF,2008/2009,65
115,1338,483,10205,2012-02-22 00:00:00,20,Slow,NaN,Little,26,Short,...,Medium,51,Press,56,Normal,Cover,10205,Villarreal CF,2008/2009,65
116,1339,483,10205,2013-09-20 00:00:00,24,Slow,NaN,Little,26,Short,...,Medium,51,Press,56,Normal,Cover,10205,Villarreal CF,2008/2009,65
117,1340,483,10205,2014-09-19 00:00:00,35,Balanced,42.0,Normal,35,Mixed,...,Medium,51,Press,56,Normal,Cover,10205,Villarreal CF,2008/2009,65
